# Approaching a 3D Hot Jupiter Model With PLUTO
## Summary of Key Attributes of the Setup
### definitions.h
- MHD physics module
- Spherical geometry
- Rotating frame
- 3 day rotation period
- Radial coordinate extends to 1.6 times the radius of Jupiter
- Unit length: $L = 1.6*6.9911\times10^9 [cm]) / 1.2$
- Unit velocity: $V = 1.0e5 [cm/s]$
- Unit density: $1.0e-9$
- $div(B) = 0$ control: constrained transport with UCT_HLL EMF average
- Background field splitting: enabled
- Resistivity: enabled, super time-stepping

Consequently, unit time is $T = L / V = 93214.6666667 [s] = 1.07887345679 [days]$

### pluto.ini
**Grid and solver**
- X1-grid (radius): 2 coordinate patches, each with with uniform spacing
    - 0.01 to 1.0 has 20 elements
    - 1.0 to 1.2 has 10 elements
- X2-grid (elevation): 1 coordinate patch with 36 elements, extending $\pm80^\circ$ above the equator
- X3-grid (azimuth): 1 coordinate patch with 80 elements, wrapping around the globe completely
- CFL = 0.25 with max var 1.1
- hllc solver
- van Leer limiter

**Boundary conditions**
- X1-beg: axisymmetric
- X1-end: userdef
- X2-beg: outflow
- X2-end: outflow
- X3-beg: periodic
- X3-end: periodic

**User parameters**
- ALPHA = 10.0
- VMAX = 1.0
    - Max value of the $V_{phi}$ profile
- EXP_DECAY = 10.0  
    - Time constant for the exponential $V_phi$ profile
- TRELAX = 8.403361344  
    - Relaxation time in code units
- BSURFACE = 5
    - Surface value of background magnetic field

### init.c
**Body force**
- The gravitational field is given by
$$
g = 
   \begin{cases} 
      -1/R^2           & \quad{\rm for} \quad R > 1 \\
      aR + bR^2 + cR^3 & \quad{\rm for} \quad R < 1 \\
   \end{cases}
$$
Additionally, the rotating frame weakens the effect of gravity in proportion to the cylindrical radius (about the axis of rotation) due to centrifugal force. PLUTO handles this internally for all dynamics, but the centrifugal force needs to be accounted for manually in initial and boundary conditions. This is still a TODO.

**Initial conditions**
- The density field is initialized to a state of hydrostatic equilibrium
- All velocities are initially 0
- Angular velocity of rotating frame is $$\Omega_z = \frac{L}{V T_r}=\frac{(1.6*6.9911\times10^9 [cm]) / 1.2}{10^5 [cm/s] \times3*34*3600 [s]}=0.357$$ in code units, where $L$ and $V$ are the unit length and velocity, respectively, and $T$ is the period of rotation. Note that we define $L$ as the planet radius, $R_p$, divided by the end coordinate of the radius grid ($1.2$) so that unit radius ($1.0$) corresponds to the gravitational field transition from the interior of the planet to the exterior, and maximum radius ($1.2$) corresponds to the edge of the atmosphere

**Boundary conditions**
- At the end of the radial coordinate, all velocities are 0
- At the end of the radial coordinate, density is set to $exp\bigg(\alpha\big((\frac{1}{R}-1) + \frac{1}{2}\Omega_z^2R^2sin^2(\theta)\big)\bigg)$ and pressure is set to $\rho/\alpha$ (isothermal). The second term in the exponential is to account for centrifugal force, and this also appears in the initial condition
- At the end of the radial coordinate, the radial component of the magnetic field is reflected

**Winds (body force along phi coordinate)**
- Winds can be enabled using an internal boundary condition, which forces a certain $v_{phi}$ profile in a portion of the fluid. Alternatively, winds can be imposed as a body force. The latter approach is preferred since it can be time-dependent in a sense by _relaxing_ to the desired profile, while the former generally has a discontinuous state at the boundary of the profile.
- Using the relaxation scheme, the force along the phi coordinate is as follows: $$F_{phi} = \frac{V_{profile} - V_{phi}}{\tau_{relax}}$$
    Where $V_{profile}$ is one of the time-independent profiles below, $V_{phi}$ is the current velocity along the phi coordinate, and $\tau = t [days] / T$. For example, a relaxation of 3 rotation periods corresponds to $\tau = 3\times3 / T = 3\times3 / 1.07887345679 = 8.342$ code units.
- Two wind profiles are available:
    - Exponential profile (`#define WINDS_EXP`): $$exp\big(10(R/R_p - 1)\big) sin(m\theta + b)$$
    - Quadratic profile (`#define WINDS_QUAD`): $$\bigg(\frac{R}{R_p}\bigg)^2sin(m\theta + b)$$
    Where $m$ and $b$ are constants used to make the profiles maximum at the equator, and $0$ at the ends of the elevation grid

**Magnetic field**
- Magnetic field splitting used, which means that a background curl-free magnetic field ($B0$) is specified, and the magnetic field recorded by PLUTO is the deviation from this background field ($B1$).
- Two background fields are available:
    - Dipole (`#define B_DIPOLE`): $$B0 = \frac{M}{r^3}\big(2cos(\theta)\hat{r} + sin(\theta)\hat{\theta}\big)$$
    - Weak uniform (`Define B_UNIFORM`): $$B0 = \frac{\sqrt{8\pi\times min(p)}}{1000} \hat{z}$$
    Where $M = \text{BSURFACE}\times max(R)^3 / (V \times \sqrt{4\pi P})$, where the last factor is for converting the field in Gauss to code units, and $min(p)$ is the minimum pressure in the domain.

# Analysis of Results

In [1]:
%config IPCompleter.greedy=True

import os
import sys
import numpy as np

import matplotlib
matplotlib.use('agg')
import imageio

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyPLUTO as pp

from __future__ import print_function

# Uncomment this to display plots in this notebook. Not recommended
# for large data sets (e.g. more than 10 frames).
# Note that all plots will be saved to the working directory anyway.
# %matplotlib inline

## Loading Data
Make sure ```path_to_script```, below, is set to the location where this script is running

In [16]:
path_to_script = os.path.join(os.environ['PLUTO_DIR'], 'Work\\')
w_dir = os.path.join(path_to_script, "data")
print("Loading data from", w_dir)

data_dirs = [directory for directory in os.listdir(os.path.join(w_dir)) if "." not in directory]
print("Found data directories: ")
i = 0
for dir in data_dirs:
    print("\t{0}: {1}".format(i, dir))
    i = i + 1

Loading data from D:\PLUTO\Work\data
Found data directories: 
	0: 10t_centrifugal_exp_winds
	1: 10t_centrifugal_exp_winds_old
	2: 10t_centrifugal_no_winds
	3: 10t_centrifugal_no_winds_old
	4: 3t_centrifugal_no_winds
	5: 3t_centrifugal_no_winds_old
	6: 3t_with_centrifugal
	7: centrifugal_exp_winds_tau2p8
	8: centrifugal_exp_winds_tau5p6
	9: centrifugal_exp_winds_tau8p4
	10: centrifugal_exp_winds_tau8p4_old
	11: centrifugal_exp_winds_vr0_vtheta0
	12: centrifugal_no_winds
	13: centrifugal_no_winds_vr0_vtheta0
	14: disk_planet_no_rotation
	15: disk_planet_rotation
	16: mag_15t
	17: mag_30t
	18: mag_3t
	19: mag_dipole
	20: mag_dipole_15t
	21: mag_dipole_1t
	22: mag_dipole_1t_10G
	23: mag_dipole_30t
	24: mag_dipole_3t
	25: mag_dipole_current
	26: mag_dipole_current_old
	27: mag_dipole_no_bg_field
	28: mag_dipole_no_bg_field_small_t
	29: mag_dipole_no_resistivity
	30: mag_dipole_reflective
	31: mag_dipole_small_t
	32: mag_new_density
	33: mag_uniform_b
	34: mag_uniform_b_high_res
	35: res_20u

Next, we may have a choice between different data sets to load. These have been enumerated above. To change the data set, change the value of `set_idx`, below to one of the other indices.

In [17]:
set_idx = 30
wdir = os.path.join(w_dir, data_dirs[set_idx] + os.sep)
print("Using working directory", wdir)

Using working directory D:\PLUTO\Work\data\mag_dipole_reflective\


Next, we use pyPLUTO to load the data in ```wdir``` into a ```pload``` object. ```dh[n]``` is the data handle for data.```n```.dbl.

In [18]:
n_frames = len([file for file in os.listdir(wdir) if "data" in file and ".dbl" in file])
dh = list() # List of data handles
for i in range(n_frames):
    d = pp.pload(i, w_dir = wdir)
    dh.append(d)

Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0000.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0001.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0002.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0003.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0004.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0005.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0006.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0007.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0008.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0009.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0010.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0011.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0012.dbl
Reading Data file : D:\PL

Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0110.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0111.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0112.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0113.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0114.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0115.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0116.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0117.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0118.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0119.dbl
Reading Data file : D:\PLUTO\Work\data\mag_dipole_reflective\data.0120.dbl


In [19]:
def load_unit_constants(w_dir):
    '''
    Loads the scaling constants from a file.

    Parameters
    ----------
    w_dir : str
        The location where the file is to be loaded from
    '''
    print("Loading unit constants")
    UNIT_DENSITY  = sys.maxint
    UNIT_VELOCITY = sys.maxint
    UNIT_LENGTH   = sys.maxint
    with open(os.path.join(w_dir, "unit_constants.dat"), "r") as f:
        for line in f:
            l = line.split(" ")
            assert(len(l) == 2), "Error reading unit_constants.dat ({0})".format(l)
            if l[0].startswith("UNIT_DENSITY"):
                UNIT_DENSITY = float(l[1])
            elif l[0].startswith("UNIT_VELOCITY"):
                UNIT_VELOCITY = float(l[1])
            elif l[0].startswith("UNIT_LENGTH"):
                UNIT_LENGTH = float(l[1])
    assert(UNIT_DENSITY != sys.maxint and UNIT_VELOCITY != sys.maxint and UNIT_LENGTH != sys.maxint)
    return UNIT_DENSITY, UNIT_VELOCITY, UNIT_LENGTH
            
try:
    UNIT_DENSITY, UNIT_VELOCITY, UNIT_LENGTH = load_unit_constants(wdir)
except IOError:
    print("Using default unit constants (not loaded from run)")
    UNIT_DENSITY  = 1.0e-9
    UNIT_VELOCITY = 1.0e5
    UNIT_LENGTH   = 1.6*6.9911e9 / 1.2

print("Using unit constants:\n\tUNIT_DENSITY = {0} [gr/cm]\n\tUNIT_VELOCITY = {1} [cm/s]\n\tUNIT_LENGTH = {2} [cm]\n".format(
        UNIT_DENSITY, UNIT_VELOCITY, UNIT_LENGTH
    )
)

def get_physical_b_units(b):
    return b * UNIT_VELOCITY * np.sqrt(4 * np.pi * UNIT_DENSITY)

for frame in dh:
    frame.bx1 = get_physical_b_units(frame.bx1)
    frame.bx2 = get_physical_b_units(frame.bx2)
    frame.bx3 = get_physical_b_units(frame.bx3)

Loading unit constants
Using unit constants:
	UNIT_DENSITY = 1e-09 [gr/cm]
	UNIT_VELOCITY = 100000.0 [cm/s]
	UNIT_LENGTH = 9321466666.7 [cm]



In [20]:
def load_user_params(w_dir):
    '''
    Loads user-defined parameters.

    Parameters
    ----------
    w_dir : str
        The location where the file is to be loaded from
    '''
    print("Loading user params")
    params = []
    with open(os.path.join(w_dir, "user_params.dat"), "r") as f:
        for line in f:
            params.append(float(line))
    return params
            
try:
    params_list = load_user_params(wdir)
    user_params = {}
    user_params['ALPHA']     = params_list[0]
    user_params['VMAX']      = params_list[1]
    user_params['EXP_DECAY'] = params_list[2]
    user_params['TRELAX']    = params_list[3]
    user_params['BSURFACE']  = params_list[4]
    print("User params:")
    for k,v in user_params.items():
        print("\t" + k + ": " + str(v))
except IOError:
    print("No user params found")
    pass

Loading user params
User params:
	VMAX: 1.0
	ALPHA: 10.0
	TRELAX: 8.403361
	EXP_DECAY: 10.0
	BSURFACE: 5.0


In [22]:
class MagField:
    def __init__(self):
        self.bx1 = 0
        self.bx2 = 0
        self.bx3 = 0
        
    def init_dipole_field(self, ref_frame, B_surface):
        '''
        Generates a dipole field with the specified surface value.

        Parameters
        ----------
        ref_frame : pyPLUTO.pload
            Reference data frame, used for getting coordinates etc.
        B_surface : double
            Value of B field at the edge of the atmosphere
        '''
        self.bx1 = np.zeros_like(ref_frame.bx1)
        self.bx2 = np.zeros_like(ref_frame.bx2)
        self.bx3 = np.zeros_like(ref_frame.bx3)
        M = B_surface * (ref_frame.x1r[-1] ** 3)
        r_tot     = ref_frame.n1_tot
        theta_tot = ref_frame.n2_tot
        phi_tot   = ref_frame.n3_tot
        for i in range(r_tot):
            r_cubed = ref_frame.x1[i] ** 3
            for j in range(theta_tot):
                bg_bx1 = 2.0 * M * np.cos(ref_frame.x2[j]) / r_cubed
                bg_bx2 = M * np.sin(ref_frame.x2[j]) / r_cubed
                for k in range(phi_tot):
                    self.bx1[i,j,k] = bg_bx1
                    self.bx2[i,j,k] = bg_bx2
    
    def load_field(self, ref_frame, w_dir, fname="bg_field.dat"):
        '''
        Loads the background field from a file. Order of iteration is k-dir, j-dir,
        i-dir, and each line contains the field components in order i-j-k.
        
        Also converts to natural units from code units.

        Parameters
        ----------
        ref_frame : pyPLUTO.pload
            Reference data frame, used for getting coordinates etc.
        w_dir : str
            The location where the file is to be loaded from
        fname : str
            The name of the file to load the background field from
        '''
        r_tot     = ref_frame.n1_tot
        theta_tot = ref_frame.n2_tot
        phi_tot   = ref_frame.n3_tot
        with open(os.path.join(w_dir, fname), "r") as f:
            data = f.readlines()
        expected_num_lines = r_tot * theta_tot * phi_tot
        assert(len(data) == expected_num_lines), "Error reading {0} (got {1} lines, expected {2})".format(fname, len(data), expected_num_lines)
        self.bx1 = np.zeros_like(ref_frame.bx1)
        self.bx2 = np.zeros_like(ref_frame.bx2)
        self.bx3 = np.zeros_like(ref_frame.bx3)
        for k in range(phi_tot):
            for j in range(theta_tot):
                for i in range(r_tot):
                    idx = k * theta_tot * r_tot + j * r_tot + i
                    B0ijk = data[idx].split(" ")
                    B0ijk = [float(e) for e in B0ijk]
                    assert(len(B0ijk) == 3), "Wrong number of field components"
                    self.bx1[i,j,k] = B0ijk[0]
                    self.bx2[i,j,k] = B0ijk[1]
                    self.bx3[i,j,k] = B0ijk[2]
        self.bx1 = get_physical_b_units(self.bx1)
        self.bx2 = get_physical_b_units(self.bx2)
        self.bx3 = get_physical_b_units(self.bx3)

try:
    print("LOADED background field")
    B0 = MagField()
    B0.load_field(dh[0], wdir)
except:
    print("No background field file to load")

LOADED background field


## Animations
Now we define some functions and variables to help with making animations from the data

In [23]:
# Image size
width = 8.5
height = 11

# One-time plot configuration
plt.rcParams['figure.figsize'] = [width, height]
plt.tight_layout()
plt.subplots_adjust(hspace=0.5)

im_set = []
DEFAULT_FRAME_PERIOD = 0.25
    
def save_plt(fname, w_dir=wdir, image_set=im_set, track=True):
    '''
    Saves plots to the working directory

    Parameters
    ----------
    fname : str
        The name to be given to the image file
    w_dir : str
        (Optional) The location where the file is to be saved
    image_set : list
        Holds the file names related to this analysis set
    track : bool
        True if this file should be tracked in the analysis set
    '''
    name = fname + '.png'
    plt.savefig(os.path.join(w_dir, name))
    if track:
        image_set.append(name)


def end_set(frame_period=DEFAULT_FRAME_PERIOD, w_dir=wdir, image_set=im_set, del_frames=True):
    # Make .gif animation of the frames
    images = []
    for filename in image_set:
        images.append(imageio.imread(os.path.join(w_dir, filename)))
    gif_name = os.path.join(w_dir, image_set[-1][:-4] + '.gif')
    print(gif_name)
    imageio.mimsave(gif_name, images, duration=frame_period)
    # Delete individual frames to save space
    if del_frames:
        for filename in image_set:
            os.remove(os.path.join(w_dir, filename))
    del image_set[:]
    
def clear_set(image_set=im_set):
    del image_set[:]

## 2D Plots
We now define some functions to help us with plotting, and also perform some one-time plot configurations.

In [24]:
def get_field_qty(field, df, bg_field=True):
    plot_v = field == 'v_phi' or field == 'v_r' or field == 'v_theta'
    plot_b = field == 'b_phi' or field == 'b_r' or field == 'b_theta'
    plot_j = field == 'j_phi' or field == 'j_r' or field == 'j_theta'
    assert(field == 'rho' or plot_v or plot_b or plot_j), "field argument is invalid"
    
    # Find the field quantity
    if field == 'rho':
        qty = df.rho
    elif plot_v:
        if field == 'v_r':
            qty = df.vx1
        elif field == 'v_theta':
            qty = df.vx2
        elif field == 'v_phi':
            qty = df.vx3
    elif plot_b:
        if field == 'b_r':
            qty = df.bx1
            if bg_field:
                qty = np.copy(qty) + B0.bx1
        elif field == 'b_theta':
            qty = df.bx2
            if bg_field:
                qty = np.copy(qty) + B0.bx2
        elif field == 'b_phi':
            qty = df.bx3
            if bg_field:
                qty = np.copy(qty) + B0.bx3
    elif plot_j:
        if field == 'j_r':
            qty = df.jx1
        elif field == 'j_theta':
            qty = df.jx2
        elif field == 'j_phi':
            qty = df.jx3
    return qty;


# TODO(Tyler): instead of simply passing in True or False, we should be able to specify
# the desired elevation angle, or azimuthal angle and then choose the closest value
# that we have on the grid
def plot_2d(
    data,
    const_elevation=False,
    theta=90,
    const_azimuth=False,
    phi=0,
    log=False,
    data_idx=-1,
    field='',
    arrows=False,
    bg_field=True,
    v_min=float('inf'),
    v_max=float('inf')
):
    '''
    Plots scalar and vector field quantities.
    
    Parameters
    ----------
    data : list of pyPLUTO.pload
        List of data frames
    const_elevation : bool
        If True, plots the density as a function of radius and elevation
        Exclusive to const_azimuth
    theta : double
        The elevation (latitude) to fix, if applicable. Given in degrees
    phi : double
        The longitude to fix, if applicable. Given in degrees
    const_azimuth : double
        If True, plots the density as a function of radius and azimuth
        Exclusive to const_elevation
    log : bool
        (Optional) Plots the logarithm for scalar fields (base 10)
    data_idx : int
        (Optional) Causes only the specified frame to be plotted
    field : str
        The field quantity to plot
            - "rho" for density
            - "v_phi" for azimuthal (zonal) velocity
            - "v_r" for radial velocity
            - "v_theta" for meridional velocity
            - And similarly for magnetic flux density (e.g. "b_phi", etc.)
            - And similarly for current (e.g. "j_theta", etc.)
    arrows : bool
        Plot field vectors if true
    bg_field : bool
        Sum the background magnetic field with the deviation field if true
    v_min : float
        Specifies what the colorbar's min color should be
    v_max : float
        Specifies what the colorbar's max color should be
    '''
    
    if const_elevation == False and const_azimuth == False:
        const_elevation = True
    
    # Sanity check on inputs. Not trying to be exhaustive here, just
    # trying to reduce the probability that I make a mistake while using
    # this code
    plot_v = field == 'v_phi' or field == 'v_r' or field == 'v_theta'
    plot_b = field == 'b_phi' or field == 'b_r' or field == 'b_theta'
    plot_j = field == 'j_phi' or field == 'j_r' or field == 'j_theta'
    assert(field == '' or plot_v or plot_b or plot_j), "field argument is invalid"
    assert(const_elevation != True or const_azimuth != True), "EITHER the azimuth or elevation may be specified, not both"
    assert(not (field == '' and log == True)), "Cannot do log plot if no field quantity is specified"
    assert(data_idx <= len(dh)), "data_idx must be <= len(dh)"
    
    single_only = (data_idx != -1)
    nframes = len(data)
    nrows   = 1 if single_only else np.round(np.sqrt(nframes))
    ncols   = 1 if single_only else np.round(np.sqrt(nframes) + 1)
    theta_idx = (np.abs(dh[0].x2 - theta * np.pi / 180)).argmin()
    phi_idx   = (np.abs(dh[0].x3 - phi * np.pi / 180)).argmin()
    fig = plt.figure()
    for idx in range(nframes):
        if single_only and idx != data_idx:
            continue
        # Set up quantities
        if plot_v:
            U = get_field_qty("v_r", data[idx], bg_field=bg_field)
        elif plot_b:
            U = get_field_qty("b_r", data[idx], bg_field=bg_field)
        elif plot_j:
            U = get_field_qty("j_r", data[idx], bg_field=bg_field)
        X = data[idx].x1
        if const_azimuth:
            Y = 90 - (180 / np.pi) * data[idx].x2
            if plot_v:
                V = get_field_qty("v_theta", data[idx], bg_field=bg_field)
            if plot_b:
                V = get_field_qty("b_theta", data[idx], bg_field=bg_field)
            if plot_j:
                V = get_field_qty("j_theta", data[idx], bg_field=bg_field)
            U = U[:,:,phi_idx].T
            V = V[:,:,phi_idx].T
            V = -1 * V # Needed for getting the theta vector direction right
            qty = get_field_qty(field, data[idx], bg_field=bg_field)[:,:,phi_idx].T
            annotations = [field, 'X1 (radius)', 'X2 (elevation)']
        elif const_elevation:
            Y = (180 / np.pi) * data[idx].x3
            if plot_v:
                V = get_field_qty("v_phi", data[idx], bg_field=bg_field)
            if plot_b:
                V = get_field_qty("b_phi", data[idx], bg_field=bg_field)
            if plot_j:
                V = get_field_qty("j_phi", data[idx], bg_field=bg_field)
            U = U[:,theta_idx,:].T
            V = V[:,theta_idx,:].T
            qty = get_field_qty(field, data[idx], bg_field=bg_field)[:,theta_idx,:].T
            annotations = [field, 'X1 (radius)', 'X3 (azimuth)']

        # Plotting
        if single_only:
            plt.subplot(nrows, ncols, 1)
        else:
            plt.subplot(nrows, ncols, idx + 1)
        if field != '':
            if log:
                annotations[0] = "Log10 " + annotations[0]
                qty = np.log10(qty)
            if v_min == float('inf'):
                v_min = np.floor(np.min(qty))
            if v_max == float('inf'):
                v_max = np.ceil(np.max(qty))
            plt.pcolormesh(X, Y, qty, vmin=v_min, vmax=v_max)
            if (single_only or ((idx != 0 and (idx+1) % ncols == 0) or idx == nframes-1)):
                plt.colorbar()
        if arrows:
            plt.quiver(X, Y, U, V)
        annotations[0] = annotations[0] +  ' (time: ' + str(np.round(data[idx].SimTime, 2)) + ')'
        plt.title(annotations[0])
        plt.xlabel(annotations[1])
        if single_only or idx % ncols == 0:
            plt.ylabel(annotations[2])
        if single_only:
            break

    # Save image to working directory
    if const_azimuth:
        fname = "rad_and_theta_vs_" + field
    elif const_elevation:
        fname = "rad_and_phi_vs_" + field
    if arrows:
        if fname[-1] != '_':
            fname = fname + "_"
        fname = fname + "vector"
    fname = fname + "_max" + str(v_max)
    if bg_field and plot_b:
        fname = fname + "_bgfield"
    if not single_only:
        save_plt(fname, track=False)
    else:
        fname = fname + "_t" + str(np.round(data[idx].SimTime, 5))
        save_plt(fname)
    plt.close();

We now fix the azimuth, and plot how the density varies with radius and elevation. The code below will either plot this relationship for only the first time snapshot, or for all time snapshots depending on whether `only_plot_first_frame`, below, is `True` or `False`.

# Vector Plots

## Analysis Parameters

In [25]:
# Plot a few specific frames. Note that -1 will plot all frames
frames = range(len(dh))

# Scale to these velocities for each frame
v_analyze = [0.1, 1.0]

# Scale to these magnetic fields for each frame
b_analyze = [5.0, 1e-2]

# Scale to these currents for each frame
j_analyze = [1.0]

## Velocities on an elevation slice

In [11]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='v_r', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

c:\python27\lib\site-packages\matplotlib\quiver.py:666: RuntimeWarning: divide by zero encountered in double_scalars
  length = a * (widthu_per_lenu / (self.scale * self.width))
c:\python27\lib\site-packages\matplotlib\quiver.py:666: RuntimeWarning: invalid value encountered in multiply
  length = a * (widthu_per_lenu / (self.scale * self.width))
c:\python27\lib\site-packages\matplotlib\quiver.py:719: RuntimeWarning: invalid value encountered in less
  short = np.repeat(length < minsh, 8, axis=1)
c:\python27\lib\site-packages\matplotlib\quiver.py:733: RuntimeWarning: invalid value encountered in less
  tooshort = length < self.minlength


D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_v_r_vector_max0.1_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_v_r_vector_max1.0_t30.0.gif


In [14]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='v_theta', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

c:\python27\lib\site-packages\matplotlib\quiver.py:666: RuntimeWarning: divide by zero encountered in double_scalars
  length = a * (widthu_per_lenu / (self.scale * self.width))
c:\python27\lib\site-packages\matplotlib\quiver.py:666: RuntimeWarning: invalid value encountered in multiply
  length = a * (widthu_per_lenu / (self.scale * self.width))
c:\python27\lib\site-packages\matplotlib\quiver.py:719: RuntimeWarning: invalid value encountered in less
  short = np.repeat(length < minsh, 8, axis=1)
c:\python27\lib\site-packages\matplotlib\quiver.py:733: RuntimeWarning: invalid value encountered in less
  tooshort = length < self.minlength


D:\PLUTO\Work\data\tmp\rad_and_theta_vs_v_theta_vector_max0.1_t4.99999.gif
D:\PLUTO\Work\data\tmp\rad_and_theta_vs_v_theta_vector_max1.0_t4.99999.gif


## Zonal velocities at equator

In [15]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        plot_2d(dh, const_elevation=True, theta=90, data_idx=idx, field='v_phi', arrows=True, v_min=-1*vmax, v_max=vmax)
    end_set()

D:\PLUTO\Work\data\tmp\rad_and_phi_vs_v_phi_vector_max0.1_t4.99999.gif
D:\PLUTO\Work\data\tmp\rad_and_phi_vs_v_phi_vector_max1.0_t4.99999.gif


# Currents

## Currents on an elevation slice

In [37]:
clear_set()
for jmax in j_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='j_r', arrows=True, v_min=-1*jmax, v_max=jmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_reflective\rad_and_theta_vs_j_r_vector_max1.0_t30.0.gif


In [38]:
clear_set()
for jmax in j_analyze:
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='j_theta', arrows=True, v_min=-1*jmax, v_max=jmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_reflective\rad_and_theta_vs_j_theta_vector_max1.0_t30.0.gif


## Zonal currents at equator

In [39]:
clear_set()
for jmax in j_analyze:
    for idx in frames:
        plot_2d(dh, const_elevation=True, theta=90, data_idx=idx, field='j_phi', arrows=True, v_min=-1*jmax, v_max=jmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_reflective\rad_and_phi_vs_j_phi_vector_max1.0_t30.0.gif


## Magnetic field on an elevation slice

In [14]:
clear_set()
for bmax in b_analyze:
    # With background field
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_r', arrows=True, v_min=-1*bmax, v_max=bmax)
    end_set()
    
    # Without background field
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_r', arrows=True, bg_field=False, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_r_vector_max5.0_bgfield_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_r_vector_max5.0_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_r_vector_max0.01_bgfield_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_r_vector_max0.01_t30.0.gif


In [15]:
clear_set()
for bmax in b_analyze:
    # With background field
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_theta', arrows=True, v_min=-1*bmax, v_max=bmax)
    end_set()
        
    # Without background field
    for idx in frames:
        plot_2d(dh, const_azimuth=True, data_idx=idx, field='b_theta', arrows=True, bg_field=False, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_theta_vector_max5.0_bgfield_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_theta_vector_max5.0_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_theta_vector_max0.01_bgfield_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_theta_vs_b_theta_vector_max0.01_t30.0.gif


## Zonal magnetic field at equator

In [16]:
clear_set()
for bmax in b_analyze:            
    # Without background field
    for idx in frames:
        plot_2d(dh, const_elevation=True, theta=20, data_idx=idx, field='b_phi', arrows=True, bg_field=False, v_min=-1*bmax, v_max=bmax)
    end_set()

D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_phi_vs_b_phi_vector_max5.0_t30.0.gif
D:\PLUTO\Work\data\mag_dipole_no_resistivity\rad_and_phi_vs_b_phi_vector_max0.01_t30.0.gif


# Other magnetic field quantities

In [26]:
# TODO(tyler): support velocity inputs (change y-label)
def plot_extrema_over_time(data, field, bg_field=False):
    fig, ax = plt.subplots()
    t = []
    field_max = []
    field_min = []
    for frame in data:
        t.append(frame.SimTime)
        qty = get_field_qty(field, frame, bg_field=bg_field)
        field_max.append(np.max(qty))
        field_min.append(np.min(qty))
    ax.scatter(t, field_max, c="blue", label="max")
    ax.scatter(t, field_min, c="red", label="min")
    ax.legend()
    plt.title(field)
    plt.xlabel('Time')
    plt.ylabel('{0} [G]'.format(field))
    fname = field + "_extrema_vs_t"
    if bg_field:
        fname = fname + "_bgfield"
    save_plt(fname)
    plt.close();

plot_extrema_over_time(dh, "b_r")
plot_extrema_over_time(dh, "b_theta")
plot_extrema_over_time(dh, "b_phi")
plot_extrema_over_time(dh, "v_phi")
plot_extrema_over_time(dh, "v_theta")
plot_extrema_over_time(dh, "v_r")

In [18]:
# max(b^2) vs t
fig = plt.figure()
t = []
b_sq_max = []

r_tot = dh[0].n1_tot
theta_tot = dh[0].n2_tot
phi_tot = dh[0].n3_tot
for frame in dh:
    t.append(frame.SimTime)
    max_val = float("-inf")
    for i in range(r_tot):
        for j in range(theta_tot):
            for k in range(phi_tot):
                b_sq = frame.bx1[i,j,k]**2 + frame.bx2[i,j,k]**2 + frame.bx3[i,j,k]**2
                if b_sq > max_val:
                    max_val = b_sq
    b_sq_max.append(max_val)

plt.scatter(t, b_sq_max)
plt.title('Dipole magnetic field')
plt.xlabel('Time')
plt.ylabel('max(b^2) [G^2]')
save_plt("b^2_max_vs_t")
plt.close();

In [19]:
# Integral of B^2 vs t
fig = plt.figure()
t = []
I_set = []

r_tot = dh[0].n1_tot
theta_tot = dh[0].n2_tot
phi_tot = dh[0].n3_tot
for frame in dh:
    I = 0
    t.append(frame.SimTime)
    for i in range(r_tot - 1):
        for j in range(theta_tot - 1):
            for k in range(phi_tot - 1):
                # Field quantity, choosing middle point in the spherical wedge
                b_r     = frame.bx1
                b_theta = frame.bx2
                b_phi   = frame.bx3
                b_r_s     = (b_r[i+1,j+1,k+1]     + b_r[i,j,k])     / 2
                b_theta_s = (b_theta[i+1,j+1,k+1] + b_theta[i,j,k]) / 2
                b_phi_s   = (b_phi[i+1,j+1,k+1]   + b_phi[i,j,k])   / 2
                b_sq      = b_r_s**2 + b_theta_s**2 + b_phi_s**2
                
                # Volume element
                r     = frame.x1
                theta = frame.x2
                phi   = frame.x3
                r_s     = (r[i + 1]     + r[i])     / 2
                theta_s = (theta[j + 1] + theta[j]) / 2
                d_r     = r[i + 1]     - r[i]
                d_theta = theta[j + 1] - theta[j]
                d_phi   = phi[k + 1]   - phi[k]
                vol = r_s**2 * np.sin(theta_s) * d_r * d_theta * d_phi
                
                # Update integral
                I = I + b_sq * vol
    I_set.append(I)

plt.scatter(t, I_set)
plt.title('Integral of B^2 over planet\'s volume vs time')
plt.xlabel('Time')
plt.ylabel('Integral of B^2 [G^2]')
save_plt("b_sq_integral_max_vs_t")
plt.close();

## Density

In [ ]:
idx = -1 # Make this negative, or omit, to plot for all time frames
plot_2d(dh, const_azimuth=True, log=True, data_idx=idx, field='rho')
save_plt('rho_vs_rad_vs_lat')

## Radial Density Profile
Here we will plot how the density varies with radius, when the elevation and azimuth are fixed. We will define this as a general-purpose function so that we can reuse it.

In [ ]:
def subplot_radius_vs_density_2d(data, theta, phi, log=False, first_only=False):
    '''
    Plots density profiles as a function of the radius for fixed theta and phi, for
    all time steps.
    
    Parameters
    ----------
    data : list of pyPLUTO.pload
        List of data frames
    theta : double
        The desired elevation angle (spherical coordinates)
    phi : double
        The desired azimuthal angle
    log : bool
        (Optional) Plots the logarithm of the density (base 10)
    first_only : bool
        (Optional) Causes only the first frame to be plotted
        
    Note that the values specified for theta and/or phi may not exist on the grid since
    it is discrete. The closest value will be used in such cases.
    '''
    
    theta_idx = (np.abs(dh[0].x2 - theta * np.pi / 180)).argmin()
    phi_idx = (np.abs(dh[0].x3 - phi * np.pi / 180)).argmin()
    
    nframes = 1 if first_only else len(data)
    nrows   = 1 if first_only else np.round(np.sqrt(nframes))
    ncols   = 1 if first_only else np.round(np.sqrt(nframes) + 1)
    for idx in range(nframes):
        plt.subplot(nrows, ncols, idx + 1)
        qty = np.log10(dh[idx].rho[:, theta_idx, phi_idx])
        plt.scatter(dh[idx].x1, qty)
        plt.title('Time: ' + str(np.round(dh[idx].SimTime, 2)) +
                  ' (theta = ' + str(np.round(dh[idx].x2[theta_idx] * 180 / np.pi, decimals=1)) +
                  ', phi = ' + str(np.round(dh[idx].x3[phi_idx] * 180 / np.pi, decimals=1)) + ')'
        )
        plt.xlabel('X1 (radius)')
        plt.ylabel('Log 10 Density')

### North Pole

In [ ]:
subplot_radius_vs_density_2d(dh, 0, 0, first_only=False);
save_plt('rho_vs_rad_north_pole_profile')

### Equator

In [ ]:
subplot_radius_vs_density_2d(dh, 90, 0, first_only=False);
save_plt('rho_vs_rad_equator_profile')

### South Pole

In [ ]:
subplot_radius_vs_density_2d(dh, 180, 0, first_only=False);
save_plt('rho_vs_rad_south_pole_profile')

## 3D Plots
First we will define some functions to help us with plotting in 3D. The scatter plots only work nicely with cartesian coordinates so we will have to convert from spherical.

In [ ]:
def sph2cart(r, theta, phi):
    '''
    Converts the inputted spherical coordinates to cartesian coordinates
    
    Parameters
    ----------
    r : double
        The radius
    theta : double
        The altitude
    phi : double
        The azimuth
    '''
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z


def sph2cart(r, theta, phi):
    '''
    Converts the inputted spherical coordinates to cartesian coordinates
    
    Parameters
    ----------
    r : double
        The radius
    theta : double
        The altitude
    phi : double
        The azimuth
    '''
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return x, y, z


def pltSphData3D(d, qty, axes, r=-1, theta=-1, phi=-1, silent=False, f=-1,
    log=False, v_min=float('inf'), v_max=float('inf'), size=20
):
    '''
    Displays a 3D scatter plot of the specified field quantity with one of the
    coordinates held constant.
    
    Parameters
    ----------
    d : pyPLUTO.pload
        Interface to data object
    qty : str
        Name of field quantity (e.g. 'rho' or 'prs')
    axes : matplotlib.axes.SubplotBase
        Plotting interface
    r, theta, phi : int
        (Optional) Fixes a grid index for the radius, elevation, or azimuth. This
        quantity will be kept constant throughout the plot. Only one may be set at
        a time. Defaults to a plot of constant radius.
    silent : bool
        Prints info messages when True, otherwise no messages are printed
    log : bool
        (Optional) Plots the logarithm of the field quantity (base 10)
    v_min : float
        Specifies the min value for the color scale
    v_max : float
        Specifies the max value for the color scale
    size : float
        Specifies point size to pass into the plotter
    
    Example
    -------
    # This will use the pload object `dh` to plot the density for a constant radius.
    The radius setting of 20 fixes the radius at dh.x1[20].
    pltSphData3D(dh, 'rho', r=20)
    '''
    
    # We default to a constant radius (specifically, the innermost radius)
    if r == -1 and phi == -1 and theta == -1:
        r = 0
    
    # Sanity checks on inputs
    assert(
        (r >= 0  and phi == -1 and theta == -1) or
        (r == -1 and phi >= 0  and theta == -1) or
        (r == -1 and phi == -1 and theta >= 0 )
    ), "Only one of r, phi, and theta can be specified"
    
    plot_v = qty == 'v_phi' or qty == 'v_r' or qty =='v_theta'
    plot_b = qty == 'b_phi' or qty == 'b_r' or qty =='b_theta'
    assert(qty == 'rho' or plot_v or plot_b), "Invalid field quantity"
    
    # Coordinates
    #    x1 = radial
    #    x2 = latitudinal (theta)
    #    x3 = longitudinal (azimuthal, phi)
    x, y, z, c = [], [], [], []
    
    num_r = d.x1.shape[0]
    num_theta = d.x2.shape[0]
    num_phi = d.x3.shape[0]
    
    # Find the field quantity
    if qty == 'rho':
        field = d.rho
    elif plot_v:
        if qty == 'v_r':
            field = d.vx1
        elif qty == 'v_theta':
            field = d.vx2
        elif qty == 'v_phi':
            field = d.vx3
    elif plot_b:
        if qty == 'b_r':
            field = d.bx1
        elif qty == 'b_theta':
            field = d.bx2
        elif qty == 'b_phi':
            field = d.bx3

    if log:
        field = np.log(field)
    
    # Plot the quantity
    if(r >= 0):
        assert(r <= num_r - 1), "r is too large (must be <= {0})".format(num_r - 1)
        r_l = d.x1[r]
        if not silent:
            print("Plotting sphere of radius {0}".format(r_l))
        for i in range(num_theta):
            theta_l = d.x2[i]
            for j in range(num_phi):
                phi_l = d.x3[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[r,i,j])
    elif(theta >= 0):
        assert(theta <= num_theta - 1), "theta is too large (must be <= {0})".format(num_theta - 1)
        theta_l = d.x2[theta]
        if not silent:
            print("Plotting cone with elevation angle {0} deg".format(theta_l * 180 / np.pi))
        for i in range(num_r):
            r_l = d.x1[i]
            for j in range(num_phi):
                phi_l = d.x3[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[i,theta,j])
    else:
        assert(phi <= num_phi - 1), "phi is too large (must be <= {0})".format(num_phi - 1)
        phi_l = d.x3[phi]
        if not silent:
            print("Plotting disk with azimuth {0} deg".format(phi_l * 180 / np.pi))
        for i in range(num_r):
            r_l = d.x1[i]
            for j in range(num_theta):
                theta_l = d.x2[j]
                x_p, y_p, z_p = sph2cart(r_l, theta_l, phi_l)
                x.append(x_p)
                y.append(y_p)
                z.append(z_p)
                c.append(field[i,j,phi])
    
    plt.xlabel('x')
    plt.ylabel('y')
    
    # Update the plot axes, but don't make them smaller
    axes.set_xlim3d(
        min(axes.get_xlim3d()[0], min(x)),
        max(axes.get_xlim3d()[1], max(x))
    )
    axes.set_ylim3d(
        min(axes.get_ylim3d()[0], min(y)),
        max(axes.get_ylim3d()[1], max(y))
    )
    axes.set_zlim3d(
        min(axes.get_zlim3d()[0], min(z)),
        max(axes.get_zlim3d()[1], max(z))
    )
    
    # Compute the max and min values used for the color
    if v_min == float('inf'):
        v_min = np.floor(np.min(field))
    if v_max == float('inf'):
        v_max = np.ceil(np.max(field))
    axes.scatter3D(x, y, z, c=c, vmin=v_min, vmax=v_max, s=size)

### Constant Azimuth

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=40, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=79, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_const_azimuth_vtheta' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()

In [ ]:
clear_set()
for bmax in b_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'b_theta', ax, size=1000, phi=40, v_min=-1*bmax, v_max=bmax)
        pltSphData3D(dh[idx], 'b_theta', ax, size=1000, phi=79, v_min=-1*bmax, v_max=bmax)
        save_plt('3d_const_azimuth_btheta' + "_max" + str(np.round(bmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()

In [ ]:
clear_set()
for bmax in b_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'b_r', ax, size=1000, phi=40, v_min=-1*bmax, v_max=bmax)
        pltSphData3D(dh[idx], 'b_r', ax, size=1000, phi=79, v_min=-1*bmax, v_max=bmax)
        save_plt('3d_const_azimuth_br' + "_max" + str(np.round(bmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()

### Combined Plots

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_phi', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vphi' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_theta', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vtheta' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()

In [ ]:
clear_set()
for vmax in v_analyze:
    for idx in frames:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, phi=10, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, phi=50, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, theta=0, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, theta=35, v_min=-1*vmax, v_max=vmax)
        pltSphData3D(dh[idx], 'v_r', ax, size=1000, r=1, v_min=-1*vmax, v_max=vmax)
        save_plt('3d_azimuth_and_elevation_vr' + "_vmax" + str(np.round(vmax, 2)) + "_f" + str(idx) + "_1000", track=(idx != -1))
        plt.close()
    end_set()